In [1]:
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Zipline, PyAlgoTrader, Catalyst, Backtrader, Vectorbt

# Simple Moving Average function
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

# SMA Cross Strategy class
class SmaCross(Strategy):
    n1 = 30
    n2 = 100

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()



In [2]:
import yfinance as yf

# Get BTC-USD data from yfinance
data = yf.download('BTC-USD')

# Code for running the backtest.
# Assumes the existence of `data` variable containing backtest data.
bt = Backtest(data, SmaCross)
stats = bt.run()


[*********************100%%**********************]  1 of 1 completed
C:\Users\alada\AppData\Local\Temp\ipykernel_29728\2896767434.py:8: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SmaCross)


In [3]:
#print(stats['_trades'].to_string())
stats


Start                     2014-09-17 00:00:00
End                       2024-05-27 00:00:00
Duration                   3540 days 00:00:00
Exposure Time [%]                   94.747247
Equity Final [$]                546645.747406
Equity Peak [$]                  617868.64975
Return [%]                        5366.457474
Buy & Hold Return [%]            15269.016933
Return (Ann.) [%]                   51.049624
Volatility (Ann.) [%]              131.284299
Sharpe Ratio                         0.388848
Sortino Ratio                        1.038267
Calmar Ratio                         0.541781
Max. Drawdown [%]                  -94.225502
Avg. Drawdown [%]                  -14.914778
Max. Drawdown Duration      727 days 00:00:00
Avg. Drawdown Duration       69 days 00:00:00
# Trades                                   30
Win Rate [%]                        46.666667
Best Trade [%]                    1113.534101
Worst Trade [%]                    -64.725432
Avg. Trade [%]                    

In [4]:
bt.plot()

c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
c:\Users\alada\AppData\Local\Programs\Python\Python312\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot

GridPlot(id='p1325', ...)

In [5]:
# Define a range of values to test for each parameter
param_grid = {'n1': range(5, 60, 5), 'n2': range(10, 90, 5)}
# Run the optimization
res = bt.optimize(**param_grid)

# Print the best results and the parameters that lead to these results
print("Best result: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

Best result:  27179.615451812744
Parameters for best result:  SmaCross(n1=10,n2=35)


In [6]:
res

Start                     2014-09-17 00:00:00
End                       2024-05-27 00:00:00
Duration                   3540 days 00:00:00
Exposure Time [%]                   98.898616
Equity Final [$]               2727961.545181
Equity Peak [$]                3247991.724869
Return [%]                       27179.615452
Buy & Hold Return [%]            15269.016933
Return (Ann.) [%]                   78.271487
Volatility (Ann.) [%]              123.261345
Sharpe Ratio                         0.635004
Sortino Ratio                        1.912594
Calmar Ratio                         1.436896
Max. Drawdown [%]                  -54.472611
Avg. Drawdown [%]                  -10.531663
Max. Drawdown Duration      962 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                  104
Win Rate [%]                        49.038462
Best Trade [%]                     226.718938
Worst Trade [%]                    -24.921626
Avg. Trade [%]                    

In [7]:
res['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2014-09-17,1.000000e+04,0.000000,NaT
2014-09-18,1.000000e+04,0.000000,NaT
2014-09-19,1.000000e+04,0.000000,NaT
2014-09-20,1.000000e+04,0.000000,NaT
2014-09-21,1.000000e+04,0.000000,NaT
...,...,...,...
2024-05-23,2.706621e+06,0.166679,NaT
2024-05-24,2.729886e+06,0.159516,NaT
2024-05-25,2.758740e+06,0.150632,NaT
2024-05-26,2.729573e+06,0.159612,NaT
